# Table 2&3 (fixed effects regression)

In [29]:
import pandas as pd
import numpy as np
import matplotlib as plt


df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

(16127, 31)


* ***Construct Work Category Dummies***

In [30]:
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#print(df.shape) to check column number (16127,36)
df.head()

,authority_code,year,auth_anno,fpsb_auction,n_bidders,discount,reserve_price,work_category,complex_work,delay_ratio,...,ctrl_exp_turin_co_sample,ctrl_pop_exp_turin_co_sample,ctrl_pop_turin_pr_sample,ctrl_exp_turin_pr_sample,ctrl_pop_exp_turin_pr_sample,OG03_dummy,OG01_dummy,OG_rest_dummy,OG_dummy,OS_dummy
0,543.0,2004.0,1642.0,0.0,3.0,8.150,2.879197e+05,OS24,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
1,543.0,2004.0,1642.0,0.0,3.0,8.150,2.879197e+05,OS24,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
2,543.0,2004.0,1642.0,0.0,3.0,8.150,2.879197e+05,OS24,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
3,3090272.0,2000.0,798.0,0.0,77.0,17.313,8.530979e+05,OG03,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1,0,0,1,0
4,3090272.0,2000.0,798.0,0.0,45.0,16.750,1.257377e+06,OG03,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1,0,0,1,0


* ***Treated vs Controls***

In [31]:
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

* ***PA Specific***

In [32]:
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#본격적으로 df에 id_auth 칼럼을 만들것임
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

* regression tools

In [33]:
#2001~2010 dummy
for j in range(2001,2011):
    df['_Iyear_'+str(j)] = 0
    for i in range(len(df)):
        if df.loc[i,'year'] == j:
            df.loc[i, '_Iyear_'+str(j)] = 1
            

In [9]:
#treatment ; "turin_co_sample turin_pr_sample" == 1
#ctrl_exp_`t' == 1 
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262
#fe_name = <fe_var>; fixed effect

SyntaxError: unexpected EOF while parsing (<ipython-input-9-7d36911b743c>, line 16)

In [38]:
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
df_reg_dis = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

In [41]:
# 1. statasmodel regression -> coeff vals/ std error dont match 
# singular matrix check; [_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']
import statsmodels.api as sm

x = df_reg_dis[['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']]
y = df_reg_dis['discount']
model = sm.OLS(y,x).fit()
predictions = model.predict(x)
model.summary()

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1827: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               discount   R-squared (uncentered):                   0.797
Model:                            OLS   Adj. R-squared (uncentered):              0.796
Method:                 Least Squares   F-statistic:                              704.2
Date:                Sun, 28 Jun 2020   Prob (F-statistic):                        0.00
Time:                        13:57:54   Log-Likelihood:                         -4275.1
No. Observations:                1262   AIC:                                      8564.
Df Residuals:                    1255   BIC:                                      8600.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
fpsb_auction    18.7845      0.809     23.232      0.000      17.198      20.371
_Iyear_2001     12.9847      0.541     23.989      0.000      11.923      14.047
_Iyear_2002     11.7676      0.492     23.917      0.000      10.802      12.733
_Iyear_2003     10.4615      0.544     19.221      0.000       9.394      11.529
_Iyear_2004     12.1567      0.483     25.163      0.000      11.209      13.104
_Iyear_2005     13.9959      0.518     27.006      0.000      12.979      15.013
_Iyear_2006     14.1079      0.640     22.031      0.000      12.852      15.364
_Iyear_2007           0          0        nan        nan           0           0
_Iyear_2008           0          0        nan        nan           0           0
_Iyear_2009           0          0        nan        nan           0           0
_Iyear_2010           0          0        nan        nan           0           0
==============================================================================
Omnibus:                       70.083   Durbin-Watson:                   1.386
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               82.280
Skew:                           0.570   Prob(JB):                     1.36e-18
Kurtosis:                       3.513   Cond. No.                          inf
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [43]:
#2 econtools
!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [54]:
mt.reg(df_reg_dis,'discount',['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006'], fe_name = 'authority_code', cluster = 'auth_anno')
#singular matrix delete['_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']
#values checked PanelA Discount(1)

Dependent variable:	discount
N:			1262
R-squared:		0.5047
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
              coeff    se      t   p>t CI_low CI_high
fpsb_auction 13.095 1.609  8.137 0.000  9.902  16.288
_Iyear_2001   2.181 1.144  1.906 0.060 -0.089   4.450
_Iyear_2002   0.583 1.140  0.512 0.610 -1.678   2.844
_Iyear_2003  -0.243 1.111 -0.219 0.827 -2.446   1.961
_Iyear_2004   1.580 1.034  1.528 0.130 -0.472   3.631
_Iyear_2005   3.764 1.155  3.259 0.002  1.473   6.056
_Iyear_2006   4.024 1.281  3.140 0.002  1.482   6.566

In [55]:
'''#3 PanelOLS
!pip install linearmodels
from linearmodels import PanelOLS
exog_vars = ['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']
dis = df_reg_dis['discount']
exog = sm.add_constant(df_reg_dis[exog_vars])
mod = PanelOLS(dis, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)'''
#y variable syntax error

"#3 PanelOLS\n!pip install linearmodels\nfrom linearmodels import PanelOLS\nexog_vars = ['fpsb_auction','_Iyear_2001','_Iyear_2002','_Iyear_2003','_Iyear_2004','_Iyear_2005','_Iyear_2006','_Iyear_2007','_Iyear_2008','_Iyear_2009','_Iyear_2010']\ndis = df_reg_dis['discount']\nexog = sm.add_constant(df_reg_dis[exog_vars])\nmod = PanelOLS(dis, exog, entity_effects=True)\nfe_res = mod.fit()\nprint(fe_res)"